In [1]:
import logging.config
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import os
import shutil
import pyodbc
import subprocess
import datetime as dt
import logging
class get_Intacct_data():
    def __init__(self,entity,sql_table_name,li_cols, **kwargs):
        self.sql_server_name = kwargs.get('sql_server_name')
        self.sql_database_name = kwargs.get('sql_database_name')
        self.sql_username = kwargs.get('sql_username')
        self.sql_password = kwargs.get('sql_password')
        self.sender_id = kwargs.get('sender_id')
        self.sender_password = kwargs.get('sender_password')
        self.company_id = kwargs.get('company_id')
        self.user_id = kwargs.get('user_id')
        self.user_password = kwargs.get('user_password')
        self.sql_table_name = sql_table_name
        self.entity = entity
        self.li_cols = li_cols
        self.url = "https://api.intacct.com/ia/xml/xmlgw.phtml"
        self.headers = {"Content-Type": "application/xml"}
        logging.basicConfig(filename='log.log', level=logging.INFO,filemode='w', format='%(asctime)s:%(levelname)s:%(message)s')
        logging.info('_________Starting Proccess________')
    def query_list(self):
        query_cols = ''
        for i in self.li_cols:
            query_cols += f'<field>{i}</field>'
        return query_cols

    def count_row_sql_table(self):
        driver = '{ODBC Driver 18 for SQL Server}' 
        try:
            connection = pyodbc.connect(
                f'DRIVER={driver};SERVER={self.sql_server_name};DATABASE={self.sql_database_name};UID={self.sql_username};PWD={self.sql_password}'
            )
            logging.info(f"Checking data in {self.sql_table_name}")
            cursor = connection.cursor()
            query = f"SELECT COUNT(*) FROM {self.sql_table_name}"
            cursor.execute(query)
            results = cursor.fetchall()
            for row in results:
                return int(row[0])
            cursor.close()
            connection.close()
        except Exception as e:
            logging.error(str(e))
            return -1
    def get_data(self, offset):
        xml_payload = f"""
        <request>
            <control>
                <senderid>{self.sender_id}</senderid>
                <password>{self.sender_password}</password>
                <controlid>control_test</controlid>
                <uniqueid>false</uniqueid>
                <dtdversion>3.0</dtdversion>
            </control>
            <operation>
                <authentication>
                    <login>
                        <userid>{self.user_id}</userid>
                        <companyid>{self.company_id}</companyid>
                        <password>{self.user_password}</password>
                    </login>
                </authentication>
                <content>
                    <function controlid="query_get_data">
                        <query>
                            <object>{self.entity}</object>
                            <select>
                                {self.query_list()}
                            </select>
                            <pagesize>1000</pagesize>
                            <offset>{offset}</offset>
                        </query>
                    </function>
                </content>
            </operation>
        </request>
        """     
        return xml_payload
    def pull_data_to_sql(self, n):
        
        file_path = os.path.join(self.entity, f"{n}.csv")
        error_log = f"{file_path}_error.log" 
        bcp_command = [
            "bcp",
            f"dbo.{self.sql_table_name}", 
            "in",
            file_path, 
            "-S", self.sql_server_name,
            "-d", self.sql_database_name,
            "-c", 
            "-t", "\t", 
            "-U", self.sql_username,
            "-P", self.sql_password,
            "-e", error_log  
        ]
        try:
            logging.info(f"Processing file: {n}.csv")
            result = subprocess.run(bcp_command, capture_output=True, text=True)
            if result.returncode == 0:
                logging.info(f"Successfully imported {n}.csv!")
                os.remove(error_log)
            else:
                logging.error(f"Failed to import {n}.csv. Check the error log: {error_log}")
                print(result.stderr)  # Error output from the command
        except Exception as e:
            logging.error(f"An error occurred while processing {n}.csv: {e}")
        return None  
    def run(self):
        offset = self.count_row_sql_table()
        match offset:
            case 0:
                logging.info(f"This first running, please wait for pull data to {self.sql_table_name}")
            case default:
                logging.info(f'Preparing to update data for {self.sql_table_name}')
        if offset != -1:
            n = 0
            numremaining = 1
            if os.path.exists(self.entity):
                shutil.rmtree(self.entity)
                logging.info(f"The folder '{self.entity}' has been deleted.")
            os.makedirs(self.entity)    
            while numremaining > 0:
                try:
                    D = {}
                    for i in self.li_cols:
                        D[i] = []
                    # Code that might raise an exception
                    response = requests.post(self.url, data=self.get_data(offset), headers=self.headers, timeout=30)
                    response.raise_for_status()  # Will raise an HTTPError for bad responses (4xx and 5xx)
                    if response.status_code == 200:
                        with open('response.txt', "w", encoding="utf-8") as file:
                            file.write(response.text)
                        offset += 1000
                        n += 1
                        root = ET.fromstring(response.text).find('.//data')
                        for child in root.findall(self.entity):
                            for i in self.li_cols:
                                D[i].append(child.find(i).text)
                        df = pd.DataFrame(D)
                        if df.shape[0] > 0:
                            df.to_csv(fr'{self.entity}\{n}.csv',sep='\t', index=False, header=False)
                            self.pull_data_to_sql(n)
                        else: 
                            logging.info('No data to update')
                        numremaining = int(root.get('numremaining'))
                    else:
                        break
                except requests.exceptions.RequestException as e: 
                    error_message = str(e) 
                    logging.error(f"An error occurred: {error_message}")
                    numremaining = -1
                    break
                except Exception as e:
                    error_message = str(e)
                    logging.error(f"An unexpected error occurred: {error_message}")
                    numremaining = -1
                    break
        return None
D = {
    "sender_id" : "eastwestcenter",
    "sender_password" : "$0EN1mp@23$!",
    "company_id" : "eastwestcenter",
    "user_id" : "silverseas",
    "user_password" : "EWCSilversea19!@",
    "sql_server_name" : "tcp:ewcdbserver.database.windows.net,1433",
    "sql_database_name" : "oendbsql",
    "sql_username" : "CloudSAcb7b8ca6",
    "sql_password" : "b13RIU@HuNt0EMRm",
}

# GLACCOUNT = get_Intacct_data('GLACCOUNT','Intacct_GLAccount',['ACCOUNTNO', 'TITLE', 'ACCOUNTTYPE', 'NORMALBALANCE', 'STATUS', 'CATEGORY'],**D).run()
# li_Department = ['DEPARTMENTID', 'RECORDNO', 'TITLE', 'PARENTKEY', 'PARENTID', 'PARENTNAME','SUPERVISORKEY','SUPERVISORID','WHENCREATED','WHENMODIFIED','SUPERVISORNAME'
#                  ,'STATUS','CUSTTITLE','CREATEDBY','MODIFIEDBY'
#                  ]
# DEPARTMENT = get_Intacct_data('DEPARTMENT','Intacct_DEPARTMENT',li_Department,**D).run()
# li_Project = ['PROJECTID', 'NAME', 'PROJECTCATEGORY', 'PROJECTSTATUS', 'STATUS', 'PARENTID','CUSTOMERID','CUSTOMERNAME','PROJECTTYPE'
#               ,'MANAGERID','MANAGERCONTACTNAME'
#                  ,'LOCATIONID'
#                  ]
# PROJECT = get_Intacct_data('PROJECT','Intacct_Project',li_Project,**D).run()
# li_Transaction = ['RECORDNO','BATCH_DATE', 'BOOKID', 'REFERENCENO', 'ENTRY_DATE', 'DOCUMENT', 'ACCOUNTNO','DEPARTMENTID','LOCATIONID','DESCRIPTION'
#               ,'AMOUNT','LOCENTITY'
#                  ,'PROJECTID','VENDORID','CLASSID','GLDIMALLOWANCE'
#                  ]
# DEPARTMENT = get_Intacct_data('GLDETAIL','Intacct_GLTransaction',li_Transaction,**D).run()
li_GLBUDGETITEM = [
    'RECORDNO','BUDGETKEY','BUDGETID','CURRENCY','SYSTEMGENERATED','PERIODKEY','ACCOUNTKEY','DEPTKEY'
    ,'LOCATIONKEY','AMOUNT','BASEDON','GROWBY','PERPERIOD','NOTE','GLACCOUNTNO','ACCT_NO','ACCOUNT_TYPE'
    ,'ACCTTITLE','NORMALBALANCE','STATISTICAL','DEPARTMENTID','DEPT_NO','DEPTITLE','LOCATIONID','LOCATION_NO'
    ,'LOCATIONTITLE','PERIODNAME','REPORTINGPERIODNAME','PSTARTDATE','PENDDATE','MEGAENTITYKEY','MEGAENTITYID'
    ,'MEGAENTITYNAME','PROJECTDIMKEY','PROJECTID','PROJECTNAME','CUSTOMERDIMKEY','CUSTOMERID','CUSTOMERNAME','VENDORDIMKEY','VENDORID','VENDORNAME'
    ,'EMPLOYEEDIMKEY','EMPLOYEEID','EMPLOYEENAME','ITEMDIMKEY','ITEMID','ITEMNAME','CLASSDIMKEY','CLASSID','CLASSNAME'
    ,'GLDIMALLOWANCE','RGLDIM101000000023504_10166','EXTERNALID'
]
GLBUDGETITEM = get_Intacct_data('GLBUDGETITEM','GLBUDGETITEM',li_GLBUDGETITEM,**D).run()

KeyboardInterrupt: 

In [20]:
import logging

# Clear all handlers
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Reconfigure logging
logging.basicConfig(
    filename='log.log',  # New log file
    level=logging.DEBUG,     # Adjust logging level as needed
    filemode='w',            # Overwrite or append
    format='%(asctime)s - %(levelname)s - %(message)s'
)